In [1]:
import pandas as pd
import pathlib
import win32com.client as win32
import locale
import math

# Leitura dos arquivos:
email_df = pd.read_excel(r'Base de Dados\Emails.xlsx')
lojas_df = pd.read_csv(r'Base de Dados\Lojas.csv', sep=';', encoding='latin-1')
vendas_df = pd.read_excel(r'Base de Dados\Vendas.xlsx')

completo_df = vendas_df.merge(lojas_df, on='ID Loja')

display(completo_df)

,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final,Loja
0,1,2019-01-01,1,Sapato Estampa,1,358,358,Iguatemi Esplanada
1,1,2019-01-01,1,Camiseta,2,180,360,Iguatemi Esplanada
2,1,2019-01-01,1,Sapato Xadrez,1,368,368,Iguatemi Esplanada
3,21,2019-01-02,1,Camisa Gola V Listrado,2,116,232,Iguatemi Esplanada
4,34,2019-01-02,1,Sapato Listrado,1,363,363,Iguatemi Esplanada
...,...,...,...,...,...,...,...,...
100994,69846,2019-12-25,8,Short Estampa,2,96,192,Salvador Shopping
100995,69846,2019-12-25,8,Tênis Estampa,5,256,1280,Salvador Shopping
100996,69850,2019-12-25,8,Calça Estampa,4,177,708,Salvador Shopping
100997,69972,2019-12-26,8,Terno Liso,3,720,2160,Salvador Shopping


In [2]:
#Verificando os dados:
completo_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100999 entries, 0 to 100998
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Código Venda    100999 non-null  int64         
 1   Data            100999 non-null  datetime64[ns]
 2   ID Loja         100999 non-null  int64         
 3   Produto         100999 non-null  object        
 4   Quantidade      100999 non-null  int64         
 5   Valor Unitário  100999 non-null  int64         
 6   Valor Final     100999 non-null  int64         
 7   Loja            100999 non-null  object        
dtypes: datetime64[ns](1), int64(5), object(2)
memory usage: 6.9+ MB


In [3]:
#Separando o dataframe de cada loja:
dic_lojas = {}

for loja in lojas_df['Loja']:
    dic_lojas[loja] = completo_df[completo_df['Loja'] == loja]

In [4]:
#Pegando o último dia da planilha:
dia_indicador = completo_df['Data'].max()

In [5]:
#Salvando os dataframes de cada loja em sua pasta de Backup:
caminho = pathlib.Path('Backup Arquivos Lojas')

for loja in dic_lojas:
    pasta_loja = caminho / f'{loja}'
    pasta_loja.mkdir(exist_ok=True)

    nome_arquivo = f'{dia_indicador.day}_{dia_indicador.month}_{dia_indicador.year}_{loja}.xlsx'
    local_arquivo = pasta_loja / nome_arquivo
    dic_lojas[loja].to_excel(local_arquivo)

In [6]:
#Metas:
META_FATURAMENTO_DIA = 1000
META_FATURAMENTO_ANO = 1650000
META_QTDE_PRODUTOS_DIA = 4
META_QTDE_PRODUTOS_ANO = 120
META_TICKETMEDIO_DIA = 500
META_TICKETMEDIO_ANO= 500

In [7]:
# Define a localidade como Brasil
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

def formatar_numero(numero):
    '''Formata os valores para o padrão de moeda PT-BR'''
    return locale.format_string('%.2f', numero, grouping=True)

def verificar_cor(indicativo, meta):
    if indicativo >= meta:
        return 'green'
    else:
        return 'red'

for loja in dic_lojas:

    vendas_loja = dic_lojas[loja]
    vendas_loja_dia = vendas_loja[vendas_loja['Data']==dia_indicador]
    
    #Faturamento:
    faturamento_ano = vendas_loja['Valor Final'].sum()
    faturamento_dia = vendas_loja_dia['Valor Final'].sum()

    #Diversidade de Produtos:
    qtde_produtos_ano = len(vendas_loja['Produto'].unique())
    qtde_produtos_dia = len(vendas_loja_dia['Produto'].unique())
    
    #Ticket Médio:
    valor_venda = vendas_loja.groupby('Código Venda').sum()
    ticket_medio_ano = valor_venda['Valor Final'].mean()
    if math.isnan(ticket_medio_ano):
        ticket_medio_ano = 0
    
    valor_venda_dia = vendas_loja_dia.groupby('Código Venda').sum()
    ticket_medio_dia = valor_venda_dia['Valor Final'].mean()
    if math.isnan(ticket_medio_dia):
        ticket_medio_dia = 0

    #Criando e Enviando e-mail:
    outlook = win32.Dispatch('outlook.application')

    nome = email_df.loc[email_df['Loja']==loja, 'Gerente'].values[0]
    mail = outlook.CreateItem(0)
    mail.To = email_df.loc[email_df['Loja']==loja, 'E-mail'].values[0]
    mail.Subject = f'OnePage Dia {dia_indicador.day}/{dia_indicador.month}/{dia_indicador.year} - Loja {loja}'

    #Definindo as cores dos indicadores:
    cor_fat_ano = verificar_cor(faturamento_ano, META_FATURAMENTO_ANO)
    cor_fat_dia = verificar_cor(faturamento_dia, META_FATURAMENTO_DIA)
    cor_qtde_ano = verificar_cor(qtde_produtos_ano, META_QTDE_PRODUTOS_ANO)
    cor_qtde_dia = verificar_cor(qtde_produtos_dia, META_QTDE_PRODUTOS_DIA)
    cor_ticket_ano = verificar_cor(ticket_medio_ano, META_TICKETMEDIO_ANO)
    cor_ticket_dia = verificar_cor(ticket_medio_dia, META_TICKETMEDIO_DIA)

    #Corpo do e-mail:
    mail.HTMLBody = f'''
    <p>Bom dia, prezado(a) Sr.(a) {nome}</p>

    <p>O resultado de ontem <strong>({dia_indicador.day}/{dia_indicador.month})</strong> da <strong>Loja {loja}</strong> foi:</p>

    <table>
      <tr>
        <th>Indicador</th>
        <th>Valor Dia</th>
        <th>Meta Dia</th>
        <th>Cenário Dia</th>
      </tr>
      <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R${formatar_numero(faturamento_dia)}</td>
        <td style="text-align: center">R${formatar_numero(META_FATURAMENTO_DIA)}</td>
        <td style="text-align: center"><font color="{cor_fat_dia}">◙</font></td>
      </tr>
      <tr>
        <td>Diversidade de Produtos</td>
        <td style="text-align: center">{qtde_produtos_dia}</td>
        <td style="text-align: center">{META_QTDE_PRODUTOS_DIA}</td>
        <td style="text-align: center"><font color="{cor_qtde_dia}">◙</font></td>
      </tr>
      <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center">R${formatar_numero(ticket_medio_dia)}</td>
        <td style="text-align: center">R${formatar_numero(META_TICKETMEDIO_DIA)}</td>
        <td style="text-align: center"><font color="{cor_ticket_dia}">◙</font></td>
      </tr>
    </table>
    <br>
    <table>
      <tr>
        <th>Indicador</th>
        <th>Valor Ano</th>
        <th>Meta Ano</th>
        <th>Cenário Ano</th>
      </tr>
      <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R${formatar_numero(faturamento_ano)}</td>
        <td style="text-align: center">R${formatar_numero(META_FATURAMENTO_ANO)}</td>
        <td style="text-align: center"><font color="{cor_fat_ano}">◙</font></td>
      </tr>
      <tr>
        <td>Diversidade de Produtos</td>
        <td style="text-align: center">{qtde_produtos_ano}</td>
        <td style="text-align: center">{META_QTDE_PRODUTOS_ANO}</td>
        <td style="text-align: center"><font color="{cor_qtde_ano}">◙</font></td>
      </tr>
      <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center">R${formatar_numero(ticket_medio_ano)}</td>
        <td style="text-align: center">R${formatar_numero(META_TICKETMEDIO_ANO)}</td>
        <td style="text-align: center"><font color="{cor_ticket_ano}">◙</font></td>
      </tr>
    </table>

    <p>Segue em anexo a planilha com todos os dados para mais detalhes.</p>

    <p>Qualquer dúvida estou à disposição,</p>
    <p>Att., Diego Monutti</p>
    '''

    #Add anexos:
    attachment = pathlib.Path.cwd() / local_arquivo
    mail.Attachments.Add(str(attachment))
    mail.Send()
    print(f'E-mail da Loja {loja} enviado')

E-mail da Loja Iguatemi Esplanada enviado
E-mail da Loja Shopping Midway Mall enviado
E-mail da Loja Norte Shopping enviado
E-mail da Loja Shopping Iguatemi Fortaleza enviado
E-mail da Loja Shopping União de Osasco enviado
E-mail da Loja Shopping Center Interlagos enviado
E-mail da Loja Rio Mar Recife enviado
E-mail da Loja Salvador Shopping enviado
E-mail da Loja Rio Mar Shopping Fortaleza enviado
E-mail da Loja Shopping Center Leste Aricanduva enviado
E-mail da Loja Ribeirão Shopping enviado
E-mail da Loja Shopping Morumbi enviado
E-mail da Loja Parque Dom Pedro Shopping enviado
E-mail da Loja Bourbon Shopping SP enviado
E-mail da Loja Palladium Shopping Curitiba enviado
E-mail da Loja Passei das Águas Shopping enviado
E-mail da Loja Center Shopping Uberlândia enviado
E-mail da Loja Shopping Recife enviado
E-mail da Loja Shopping Vila Velha enviado
E-mail da Loja Shopping SP Market enviado
E-mail da Loja Shopping Eldorado enviado
E-mail da Loja Shopping Ibirapuera enviado
E-mail da L

In [9]:
def salvar_ranking_lojas(df, caminho_arquivo):
    '''Classifica e salva o ranking de faturamento de lojas em um arquivo Excel'''
    df = df.sort_values(by='Valor Final', ascending=False)
    df['Valor Final'] = df['Valor Final'].map(formatar_numero)
    df.to_excel(caminho_arquivo, index=True, header=True)
    print(f'Ranking de lojas salvo em {caminho_arquivo}')

#Faturamento anual:
faturamento_lojas_ano = completo_df.groupby('Loja')[['Loja', 'Valor Final']].sum()

#Salvar o ranking de faturamento anual:
nome_arquivo_ano = f'{dia_indicador.day}_{dia_indicador.month}_{dia_indicador.year}_Ranking Anual.xlsx'
caminho_arquivo_ano = caminho / nome_arquivo_ano
salvar_ranking_lojas(faturamento_lojas_ano, caminho_arquivo_ano)

#Faturamento último dia:
vendas_dia = completo_df[completo_df['Data']==dia_indicador]
faturamento_lojas_dia = vendas_dia.groupby('Loja')[['Loja', 'Valor Final']].sum()

#Incluir no ranking as lojas que não venderam no último dia:
faturamento_lojas_dia = faturamento_lojas_dia.merge(lojas_df[['Loja']], on='Loja', how='right')
faturamento_lojas_dia.fillna(0, inplace=True)
faturamento_lojas_dia = faturamento_lojas_dia.set_index('Loja')

#Salvar o ranking de faturamento do último dia:
nome_arquivo_dia = f'{dia_indicador.day}_{dia_indicador.month}_{dia_indicador.year}_Ranking Diário.xlsx'
caminho_arquivo_dia = caminho / nome_arquivo_dia
salvar_ranking_lojas(faturamento_lojas_dia, caminho_arquivo_dia)

Ranking de lojas salvo em Backup Arquivos Lojas\26_12_2019_Ranking Anual.xlsx
Ranking de lojas salvo em Backup Arquivos Lojas\26_12_2019_Ranking Diário.xlsx
